In [1]:
!pip install transformers==4.2.0
#!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.7-cp36-cp36m-linux_x86_64.whl

     |████████████████████████████████| 1.8 MB 1.3 MB/s eta 0:00:01
  Attempting uninstall: transformers
    Found existing installation: transformers 4.2.2
    Uninstalling transformers-4.2.2:
      Successfully uninstalled transformers-4.2.2


In [2]:
from transformers import BertForSequenceClassification, DistilBertTokenizerFast, AdamW, get_linear_schedule_with_warmup
import pandas as pd
import torch
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import time
import pickle

In [3]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(DEVICE)

cuda


In [4]:
def read_train_and_test(train_path, test_path): #path: path of the whole dataset
  #try:
    train = pd.read_csv(train_path)
    test = pd.read_csv(test_path)
  #except FileNotFoundError:
    #data_csv = pd.read_csv(path, names=['label','tweet'],header=0)
    #train=data_csv.sample(frac=0.8,random_state=200) #random state is a seed value
    #test=data_csv.drop(train.index)
    #train.to_csv('train.csv')
    #test.to_csv('test.csv')
    return train, test

train_path = "../input/data-without-bad-words/train_blank.csv"
test_path = "../input/data-without-bad-words/test_blank.csv"
train_csv, test_csv = read_train_and_test(train_path, test_path)

In [5]:
print(train_csv.tweet.values[:10])

['# anywere & # 8220 ; @ TheCooleyShow : LA = palm trees and great weather Any were else = horrible weather and no   & # 8221 ;'
 'RT @ OfficialA1King : The face you make when you see a   trying to preach on twitter http : //t.co/2T1UkUDQBw'
 '  get off my twitter   & # 128074 ;'
 'I can taste loud n   on my   & # 128541 ;'
 'Diabetes galore & # 128514 ; & # 128514 ; & # 128514 ; & # 128514 ; & # 128514 ; RT @ TIME : Colorado health officials recommend   brownie ban http : //t.co/Z59oy20TMp'
 '@ LaneBelgarde @ A7XDemery you     didnt even invite me'
 'RT @ djMemphis10 : Never hurt your Girlfriend to make a   happy & # 128175 ; # djmemphis10'
 'This is why nothing gets done .   like Coburn trying 2 ad a   amendment 2 a public works bill . Push back on the # teabagger # morningjoe'
 'RT @ ccancel14 : Miserable   talk the most   & # 128514 ; & # 9996 ; & # 65039 ;'
 'Its gorg out ! RT @ @ anggxo : This   thinks its june or something @ BriiiXO http : //t.co/mFVCmym']


In [6]:
def run_tokenizer(train_csv, test_csv, merge_label=False):
    tokenizer = DistilBertTokenizerFast.from_pretrained('bert-base-uncased') 

    def get_max_len(tokenizer, train_csv):
        tweets = train_csv.tweet.values
        max_length = 0
        for t in tweets:
          ids = tokenizer.encode(t)
          max_length = max(len(ids),max_length)
        return max_length

    max_length = get_max_len(tokenizer, train_csv)
    train_tweets, train_labels = train_csv.tweet.values, train_csv.label.values
    test_tweets, test_labels = test_csv.tweet.values, test_csv.label.values
    if merge_label == True:
       train_labels = [l if l ==0 else 1 for l in train_labels]
       test_labels = [l if l ==0 else 1 for l in test_labels]

    def tokenize_for_tweet(tokenizer, tweets, labels):
        input_ids = []
        attention_masks = []

        for t in tweets:
            input_dict = tokenizer.encode_plus(t, add_special_tokens=True, max_length=max_length, truncation=True, padding='max_length',return_tensors='pt')
            input_ids.append(input_dict['input_ids'])
            attention_masks.append(input_dict['attention_mask'])
        input_ids = torch.cat(input_ids,dim=0)
        attention_masks = torch.cat(attention_masks,dim=0)
        labels=torch.tensor(labels)
        dataset = TensorDataset(input_ids, attention_masks, labels)
        return dataset
        
    train_dataset = tokenize_for_tweet(tokenizer, train_tweets, train_labels)
    test_dataset = tokenize_for_tweet(tokenizer, test_tweets, test_labels)
    num_label = 3 if merge_label == False else 2
    return train_dataset, test_dataset, num_label

train_dataset, test_dataset, num_label = run_tokenizer(train_csv, test_csv, merge_label = True)

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

In [7]:
batch_size = 10

train_dataloader = DataLoader(train_dataset, sampler = RandomSampler(train_dataset), batch_size = batch_size)

test_dataloader = DataLoader(test_dataset, sampler = SequentialSampler(test_dataset), batch_size = batch_size)

In [8]:
bert_model = BertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels = num_label)
bert_model = bert_model.to(DEVICE)

Downloading:   0%|          | 0.00/442 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing BertForSequenceClassification: ['distilbert.embeddings.word_embeddings.weight', 'distilbert.embeddings.position_embeddings.weight', 'distilbert.embeddings.LayerNorm.weight', 'distilbert.embeddings.LayerNorm.bias', 'distilbert.transformer.layer.0.attention.q_lin.weight', 'distilbert.transformer.layer.0.attention.q_lin.bias', 'distilbert.transformer.layer.0.attention.k_lin.weight', 'distilbert.transformer.layer.0.attention.k_lin.bias', 'distilbert.transformer.layer.0.attention.v_lin.weight', 'distilbert.transformer.layer.0.attention.v_lin.bias', 'distilbert.transformer.layer.0.attention.out_lin.weight', 'distilbert.transformer.layer.0.attention.out_lin.bias', 'distilbert.transformer.layer.0.sa_layer_norm.weight', 'distilbert.transformer.layer.0.sa_layer_norm.bias', 'distilbert.transformer.layer.0.ffn.lin1.weight', 'distilbert.transformer.layer.0.ffn.lin1.bias', 'distilbert.transformer.layer.0.

In [9]:
def evaluate(model, best_acc, test_dataloader):
    num_total, num_correct = 0, 0
    model.train(False)
    with torch.no_grad():
      eval_loss = 0

      for input, mask, label in test_dataloader:
          model.zero_grad()
          
          input = input.to(DEVICE)
          mask = mask.to(DEVICE)
          label = label.to(DEVICE)

          loss, output = model(input_ids=input, attention_mask=mask, labels=label, return_dict=False)

          predict_label = torch.argmax(output, dim=1)

          num_correct += (predict_label == label).sum().item()
          num_total += len(label)

          eval_loss += loss.item()

      acc = num_correct/num_total
      if acc > best_acc:
        best_acc = acc
        torch.save(model, 'model'+str(best_acc))

    print('eval_loss: {}, accuracy: {}'.format(eval_loss,acc))
    return best_acc


In [10]:
def training(model, epochs, optimizer, train_dataloader, test_dataloader, scheduler=None):
    best_acc = 0
    for e in range(epochs):
        print('training {} epoch...'.format(e+1))
        start_time = time.time()

        train_loss =0

        model.train(True)
        for input, mask, label in train_dataloader:
            input = input.to(DEVICE)
            mask = mask.to(DEVICE)
            label=label.to(DEVICE)
            
            model.zero_grad()

            loss, output = model(input_ids=input, attention_mask=mask, labels=label,return_dict=False)

            train_loss += loss.item()
            loss.backward()

            #torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    
            optimizer.step()

            if scheduler:
              scheduler.step()

        sec = time.time()-start_time
        print('{} seconds used......'.format(sec))
        print("{} training finished! train loss: {}".format(e+1,train_loss))
        print('evaluating...')
        best_acc = evaluate(model, best_acc, test_dataloader)

In [11]:
optimizer = AdamW(bert_model.parameters(),lr = 2e-5, eps = 1e-8)

In [12]:
epochs = 10

total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = 0, num_training_steps = total_steps)

In [ ]:
training(bert_model, epochs, optimizer, train_dataloader, test_dataloader)

In [13]:
def predicting_labels(model, test_dataloader):
    true_labels, predict_labels = [], []
    for input, mask, label in test_dataloader:
        model.zero_grad()
        input=input.to(DEVICE)
        mask=mask.to(DEVICE)
        label=label.to(DEVICE)
        loss, logits = model(input_ids=input, attention_mask=mask, labels=label, return_dict=False)
        predict_label = torch.argmax(logits, dim=1)
        true_labels+=(label.tolist())
        predict_labels+=(predict_label.tolist())
    return true_labels, predict_labels

In [ ]:
model = torch.load('../input/remove-swear-with-blank-model/model0.903772443009885')
true_labels, predict_labels = predicting_labels(model, test_dataloader)

In [ ]:
accuracy=accuracy_score(true_labels,predict_labels)
print(accuracy)
pre, re, f1, sup = precision_recall_fscore_support(true_labels,predict_labels)
print(pre,re,f1,sup)

In [14]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

from sklearn.metrics import classification_report
import numpy as np
import random

In [15]:
def plot_confusion_matrix(y_true, y_pred, classes, normalize=False, title=None, cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    #classes = classes[unique_labels(y_true, y_pred)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax

In [16]:
cm = confusion_matrix(true_labels, predict_labels)
plot_confusion_matrix(true_labels, predict_labels, classes=['hate','no-hate'] ,title='Confusion matrix without rebalance')

#Plot normalized confusion matrix
plot_confusion_matrix(true_labels, predict_labels, classes=['hate','no-hate'], normalize=True,title='Normalized confusion matrix without rebalance')

plt.show()

print(classification_report(true_labels, predict_labels, target_names=['hate','no-hate']))

NameError: name 'true_labels' is not defined

In [17]:
def rebalance_dataset(dataset):
    dataset = sorted(dataset, key=lambda x:x[2])
    num_label_1 = len([inst for inst in dataset if inst[2]==1])
    num_label_0 = len([inst for inst in dataset if inst[2]==0]) 
    balanced_dataset = dataset[:num_label_0] + dataset[num_label_1:num_label_0 + num_label_1]
    random.shuffle(balanced_dataset)
    return balanced_dataset

In [18]:
re_train_dataset = rebalance_dataset(train_dataset)
re_test_dataset = rebalance_dataset(test_dataset)

In [19]:
re_train_dataloader = DataLoader(re_train_dataset, sampler = RandomSampler(re_train_dataset), batch_size = batch_size)
re_test_dataloader = DataLoader(test_dataset, sampler = RandomSampler(test_dataset), batch_size = 128)

In [ ]:
training(bert_model, epochs, optimizer, re_train_dataloader, re_test_dataloader)

In [ ]:
model2 = torch.load('../input/remove-swear-with-blank-model/model0.903772443009885')
true_labels2, predict_labels2 = predicting_labels(model2, re_test_dataloader)

In [ ]:
accuracy2=accuracy_score(true_labels2,predict_labels2)
print(accuracy2)
pre2, re2, f1_2, sup2 = precision_recall_fscore_support(true_labels2,predict_labels2)
print(pre2,re2,f1_2,sup2)

In [ ]:
cm = confusion_matrix(true_labels2, predict_labels2)
plot_confusion_matrix(true_labels2, predict_labels2, classes=['hate','no-hate'] ,title='Confusion matrix with rebalance')

#Plot normalized confusion matrix
plot_confusion_matrix(true_labels2, predict_labels2, classes=['hate','no-hate'], normalize=True,title='Normalized confusion matrix with rebalance')

plt.show()

print(classification_report(true_labels2, predict_labels2, target_names=['hate','no-hate']))